## Disclaimer: this code is <em>old</em>

<img src='cards.png' align = 'left' width = 400>

In [2]:
import numpy as np
import cv2

##### Loading the file

In [3]:
src = cv2.imread('cards.png')

# Show source image
cv2.imshow('Source Image', src)
cv2.waitKey(0)
cv2.destroyAllWindows()

##### Applying black background

In [4]:
src[np.all(src == 255, axis=2)] = 0

# Show output image
cv2.imshow('Black Background Image', src)
cv2.waitKey(0)
cv2.destroyAllWindows()

##### Applying laplacian filter

In [5]:
# Create a kernel that we will use to sharpen our image
# an approximation of second derivative, a quite strong kernel
kernel = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], dtype=np.float32)


# do the laplacian filtering as it is
# well, we need to convert everything in something more deeper then CV_8U
# because the kernel has some negative values,
# and we can expect in general to have a Laplacian image with negative values
# BUT a 8bits unsigned int (the one we are working with) can contain values from 0 to 255
# so the possible negative number will be truncated
imgLaplacian = cv2.filter2D(src, cv2.CV_32F, kernel)
sharp = np.float32(src)
imgResult = sharp - imgLaplacian

# convert back to 8bits gray scale
imgResult = np.clip(imgResult, 0, 255)
imgResult = imgResult.astype('uint8')
imgLaplacian = np.clip(imgLaplacian, 0, 255)
imgLaplacian = np.uint8(imgLaplacian)

#cv.imshow('Laplace Filtered Image', imgLaplacian)
cv2.imshow('New Sharped Image', imgResult)
cv2.waitKey(0)
cv2.destroyAllWindows()

##### Grayscale and Binary

In [6]:
# Create binary image from source image
bw = cv2.cvtColor(imgResult, cv2.COLOR_BGR2GRAY)
_, bw = cv2.threshold(bw, 40, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
cv2.imshow('Binary Image', bw)
cv2.waitKey(0)
cv2.destroyAllWindows()

##### Distance Transform

In [7]:
# Perform the distance transform algorithm
dist = cv2.distanceTransform(bw, cv2.DIST_L2, 3)

# Normalize the distance image for range = {0.0, 1.0}
# so we can visualize and threshold it
cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
cv2.imshow('Distance Transform Image', dist)
cv2.waitKey(0)
cv2.destroyAllWindows()

##### Threshold

In [8]:
# Threshold to obtain the peaks
# This will be the markers for the foreground objects
_, dist = cv2.threshold(dist, 0.4, 1.0, cv2.THRESH_BINARY)

# Dilate a bit the dist image
kernel1 = np.ones((3,3), dtype=np.uint8)
dist = cv2.dilate(dist, kernel1)
cv2.imshow('Peaks', dist)
cv2.waitKey(0)
cv2.destroyAllWindows()

##### Create and apply markers

In [11]:
# Create the CV_8U version of the distance image
# It is needed for findContours()
dist_8u = dist.astype('uint8')

# Find total markers
_, contours, __ = cv2.findContours(dist_8u, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create the marker image for the watershed algorithm
markers = np.zeros(dist.shape, dtype=np.int32)

# Draw the foreground markers
for i in range(len(contours)):
    cv.drawContours(markers, contours, i, (i+1), -1)
    
# Draw the background marker
cv2.circle(markers, (5,5), 3, (255,255,255), -1)
markers_8u = (markers * 10).astype('uint8')
cv2.imshow('Markers', markers_8u)
cv2.waitKey(0)
cv2.destroyAllWindows()

ValueError: not enough values to unpack (expected 3, got 2)

In [10]:
# Perform the watershed algorithm
cv2.watershed(imgResult, markers)

#mark = np.zeros(markers.shape, dtype=np.uint8)
mark = markers.astype('uint8')
mark = cv2.bitwise_not(mark)

# uncomment this if you want to see how the mark
# image looks like at that point
#cv.imshow('Markers_v2', mark)
# Generate random colors
colors = []
for contour in contours:
    colors.append((rng.randint(0,256), rng.randint(0,256), rng.randint(0,256)))
    
# Create the result image
dst = np.zeros((markers.shape[0], markers.shape[1], 3), dtype=np.uint8)

# Fill labeled objects with random colors
for i in range(markers.shape[0]):
    for j in range(markers.shape[1]):
        index = markers[i,j]
        if index > 0 and index <= len(contours):
            dst[i,j,:] = colors[index-1]
            
# Visualize the final image
cv2.imshow('Final Result', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'markers' is not defined

## Failed
### Reasons: old-fashioned code
### Alternatives: search other methods